In [ ]:
import pandas as pd
import os
# os.getenv
from dotenv import load_dotenv
import hvplot.pandas
import matplotlib.pyplot as plt
import panel as pn
import requests
import json
from sqlalchemy import create_engine
from sqlalchemy import inspect
from collect_contracts import *
from utils import *
import csv

In [ ]:
load_dotenv()

rarify_api_key = os.getenv("RARIFY_API_KEY")
database_uri = os.getenv("DATABASE_URI")
database_schema = os.getenv("DATABASE_SCHEMA")
display(type(rarify_api_key))

In [ ]:
engine = create_engine(database_uri, echo = False)
inspector = inspect(engine)

### Generate the formatted lists of collections 1 through 50 (excluding some, see the notes.txt file) so that it can be used for querying. This data is taken using the DataFrame and an SQL Query

In [ ]:
contracts = collect_top_100(rarify_api_key)
contracts

In [ ]:
# sql_query = f"""
#     SELECT contract_id, address, name, network_id, num_tokens, unique_owners 
#     FROM {database_schema}.collection
# """
# table = engine.execute(sql_query)

# contracts = create_top_100_contracts_dict(table)

In [ ]:
# Validate the contracts
for contract in contracts:
    print(contract)

### Query each collection from the API to obtain the contract address

### Create a dataframe for each of the set of collections

In [ ]:
collections_df = fetch_top_50_collections_data_api(contracts, rarify_api_key)

In [ ]:
# sql_query = f"""
#     SELECT contract_id, timestamp, min_price, avg_price, max_price, volume, num_trades, unique_buyers
#     FROM {database_schema}.trade
# """
# foo = engine.execute(sql_query)
# for r in foo:
#     print(r)
collections_df

In [ ]:
# collections_df = fetch_top_50_collections_database(engine, database_schema)


In [ ]:
copy = collections_df.copy()

In [ ]:
copy = avg_price_df(copy, contracts)
def extract_column_names(df):
    cols_list = []
    for col in df.columns:
        cols_list.append(col[0])
    return cols_list

def extract_readable_names(contracts):
    names =[]
    for col in contracts:
        name = contracts[col]['name']
        names.append(name)
    return names
readable_names = extract_readable_names(contracts)
cols_list = extract_column_names(copy)
copy.columns = readable_names  

In [ ]:
# copy = copy.drop("Sandbox's LANDs", axis=1)
rolling_avgs = copy.rolling(window=20).mean()
rolling_avgs

In [ ]:
y = pn.widgets.Select(
    name='Collection', 
    options=readable_names
)
plot = rolling_avgs.hvplot(x='time', y=y, colorbar=False, width=900, height=500, ylabel="Price (eth)")
pn.Column(pn.WidgetBox(y), plot)



### Create a pct_change dataframe for each of the set of collections we acquired

In [ ]:
valid_contracts = find_valid_contracts(collections_df, contracts)
valid_contracts




In [ ]:
all_whale_ratios = find_whale_ratio(contracts)
whale_ratios = find_whale_ratio(valid_contracts)
write_json_file(whale_ratios, 'whale_ratios.json')
whale_ratios
whale_ratios_df = pd.DataFrame(whale_ratios)


In [ ]:
bad_data_list = find_bad_data(valid_contracts, rarify_api_key)

In [ ]:
for i in bad_data_list:
    del valid_contracts[i]

In [ ]:
len(valid_contracts) * 6

In [ ]:
valid_collections_df = fetch_top_50_collections_data_api(valid_contracts, rarify_api_key)

In [ ]:
valid_collections_df

In [ ]:
pct_chg_df = find_pct_change(valid_collections_df, valid_contracts)
pct_chg_df = pct_chg_df.iloc[:, -(len(valid_contracts)):]

In [ ]:
pct_chg_df["top_collections_basket_pct_chg"] = pct_chg_df[pct_chg_df.columns].mean(axis=1)
pct_chg_df.tail()



In [ ]:
betas_df = pct_chg_df.drop(["InvisibleSociety_pct_chg", "ProjectKevins_pct_chg", "Bored Ape Chemistry Club_pct_chg", "mcgoblintownwtf_pct_chg"], axis=1)

In [ ]:
con_beta = betas_df["Doodles_pct_chg"].cov(betas_df["top_collections_basket_pct_chg"]) / betas_df["top_collections_basket_pct_chg"].var()
con_beta

In [ ]:
# Use a dataframe eliminating invisible society, projectkevins, invisible society
betas_contracts = valid_contracts.copy()
del betas_contracts["bc578ecca2115dac0c93c08674edc0c7d01fe09c"]
del betas_contracts["7a376c1f8edf7ba7eb94a7438a8313a17d830ff8"]
del betas_contracts["22c36bfdcef207f9c0cc941936eff94d4246d14a"]
del betas_contracts["c5b52253f5225835cc81c52cdb3d6a22bc3b0c93"]
# InvisibleSociety, ProjectKevins, Bored Ape Chemistry Club, mcgoblintownwtf
betas = find_beta(betas_df, betas_contracts)
betas_series = pd.Series(betas)
betas_series = betas_series.dropna()
betas_series = betas_series.sort_values() * 10000


In [ ]:
betas_series = betas_series.drop('Unstoppable Domains_beta')

In [ ]:
betas_series.to_csv('betas.csv')

In [ ]:
betas_series.tail().hvplot(kind="bar")

In [ ]:
std_devs = find_std_devs(pct_chg_df, valid_contracts)
std_devs_cor = std_devs.copy()
std_devs_copy = std_devs.dropna()
std_devs = std_devs[std_devs < 1.0]

In [ ]:
with open ('names.txt', 'w') as f:
    for i, v in enumerate(std_devs.index):
        f.write('"' + std_devs.index[i][0] + '"' + ",")
    

In [ ]:
std_devs["mean"] = std_devs.mean()
std_devs = std_devs.sort_values()

In [ ]:
std_devs.plot.bar()

In [ ]:
std_devs.values

In [ ]:
std_devs.to_csv('standard_deviations.csv')

In [ ]:
whale_ratios_cor = {}
for con in whale_ratios:
    whale_ratios_cor[whale_ratios[con]['name']] = whale_ratios[con]['whale_ratio']
whale_ratios_cor
whale_ratios_cor = pd.Series(whale_ratios_cor)

In [ ]:
import math
std_devs_lst = []
counter_lst = []
counter = 0
for i in std_devs_cor:
    if math.isnan(i):
        counter_lst.append(counter)
    else:
        std_devs_lst.append(i)
    counter += 1

std_devs_lst
counter_lst


In [ ]:
whale_ratios_lst = []
for i in whale_ratios_cor:
    whale_ratios_lst.append(i)

del whale_ratios_lst[2]
del whale_ratios_lst[11]
whale_ratios_lst

In [ ]:
avg_prices = find_avg_price(collections_df, contracts)
avg_prices = avg_prices.iloc[0]
avg_prices_lst = []
for i in avg_prices:
    avg_prices_lst.append(i)
avg_prices_lst

In [ ]:
all_whale_ratios_cor = {}
for con in all_whale_ratios:
    all_whale_ratios_cor[all_whale_ratios[con]['name']] = all_whale_ratios[con]['whale_ratio']
all_whale_ratios_cor = pd.Series(all_whale_ratios_cor)
all_whale_ratios_lst = []
for i in all_whale_ratios_cor:
    all_whale_ratios_lst.append(i)

In [ ]:
import numpy as np
del whale_ratios_lst[10]
del std_devs_lst[10]
np.corrcoef(whale_ratios_lst, std_devs_lst)

In [ ]:
pct_chg_df['top_collections_basket_pct_chg'].hvplot()